In [1]:

from src.values import *
from src.utils import *
import numpy as np 
from scipy.spatial.distance import cdist
from scipy.linalg import cholesky

a_t_ = a_t*3
N_t = np.floor(25*3/ a_t_).astype(np.int64)
N_z = len(z_values) 
print(f'number of grid in x :{N_t}, z:{N_z}, total:{N_t**2*N_z}')


number of grid in x :65, z:236, total:997100


In [ ]:

x = np.arange(N_t)* a_t_
y = np.arange(N_t)* a_t_ 
X, Y = np.meshgrid(x, y)
V_0000 = np.zeros((N_t, N_t, N_z))
V_1000 = np.zeros((N_t, N_t, N_z))

points = np.column_stack([X.ravel(), Y.ravel()]) # linearlized points 

exp_cov = cdist(points, points, 'sqeuclidean')
exp_cov = np.exp(-exp_cov/4/l_t**2)

c1 = lambda z: (E_G*a_t_)**2 * n(z)*(1-n(z))/(4*np.pi*l_t**2)* exp_cov


In [4]:

for i,z in enumerate(z_values):
    C_joint = c1(z)
    mean = np.zeros(C_joint.shape[0])  # Zero mean
    L = cholesky(C_joint + 1e-6 * np.eye(C_joint.shape[0]), lower=True)  # Cholesky decomposition    
    random_samples = L @ np.random.randn(C_joint.shape[0])
    
    V_0000[:, :, i]+= random_samples.reshape(N_t, N_t)
    V_1000[1:-1, :, i]+= l_t * (V_0000[2:, :, i] - V_0000[:-2, :, i])/a_t_
